In [104]:
import re
import string
import torch
import torch.nn.functional as F
import time


In [9]:
# import data
one = open('1.txt','r').read().split()
two = open('2.txt','r').read().split()
three = open('3.txt','r').read().split()
four = open('4.txt','r').read().split()

# one = open('1.txt','r').read().splitlines()
# two = open('2.txt','r').read().splitlines()
# three = open('3.txt','r').read().splitlines()
# four = open('4.txt','r').read().splitlines()

total = one + two + three + four
total = [x.lower() for x in total]
total = [x.translate(str.maketrans('','',string.punctuation)) for x in total]

# test_str = test_str.translate
#     (str.maketrans('', '', string.punctuation))

In [32]:
vocab = sorted(list(set(total)))

In [86]:
WordsToIndex = {s:i+1 for i,s in enumerate(vocab)}
IndexToWords = {i:s for s,i in WordsToIndex.items()}
IndexToWords[0] = ''
print(f"WordsToIndex: {WordsToIndex}")
print(f"IndexToWords: {IndexToWords}")

WordsToIndex: {'': 1, '0': 2, '01': 3, '07': 4, '08': 5, '1': 6, '100': 7, '101': 8, '102': 9, '104': 10, '105': 11, '106': 12, '107': 13, '108': 14, '11': 15, '110': 16, '111': 17, '112': 18, '113': 19, '114': 20, '115': 21, '116': 22, '117': 23, '118': 24, '11h': 25, '12': 26, '121': 27, '122': 28, '123': 29, '1230': 30, '124': 31, '125': 32, '126': 33, '127': 34, '128': 35, '1289': 36, '129': 37, '1296': 38, '13': 39, '130': 40, '131': 41, '132': 42, '133': 43, '134': 44, '135': 45, '136': 46, '137': 47, '138': 48, '14': 49, '140': 50, '141': 51, '142': 52, '143': 53, '144': 54, '145': 55, '146': 56, '1473': 57, '148': 58, '149': 59, '1492': 60, '15': 61, '150': 62, '151': 63, '152': 64, '1520': 65, '1522': 66, '154': 67, '157': 68, '158': 69, '159': 70, '16': 71, '160': 72, '161': 73, '1612': 74, '162': 75, '163': 76, '1637': 77, '164': 78, '165': 79, '166': 80, '167': 81, '168': 82, '169': 83, '17': 84, '170': 85, '1709': 86, '171': 87, '1722': 88, '173': 89, '176': 90, '177': 91,

In [51]:
def build_dataset(total):
    block_size = 3
    X, Y = [], []

    for i in range(len(total)-block_size):    
    # for i in range(100):
        context = total[i: i+block_size]
        target = total[i+block_size]
        # print(f"Context:{context}. Target: {target}")
        X.append([WordsToIndex[word] for word in context])
        Y.append(WordsToIndex[target])

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(total)
n1 = int(0.8*len(total))
n2 = int(0.9*len(total))

Xtr, Ytr = build_dataset(total[:n1])
Xdev, Ydev = build_dataset(total[n1:n2])
Xte, Yte = build_dataset(total[n2:])

In [52]:
Xtr.shape, Ytr.shape, Xdev.shape, Ydev.shape, Xte.shape, Yte.shape

(torch.Size([373956, 3]),
 torch.Size([373956]),
 torch.Size([46742, 3]),
 torch.Size([46742]),
 torch.Size([46742, 3]),
 torch.Size([46742]))

In [53]:
g = torch.Generator().manual_seed(2147483647) #random seed
C = torch.randn((len(vocab),2), generator=g) #embedding space for all vocab
W1 = torch.randn((6,300), generator=g) #6 -> 3 2x2 embeddings, 300 neuron hyperparameter
b1 = torch.randn(300, generator=g)
W2 = torch.randn((300,len(vocab)), generator=g)
b2 = torch.randn(len(vocab), generator=g)
parameters = [C, W1, b1, W2, b2]

In [54]:
sum(p.nelement() for p in parameters)

4842828

In [55]:
for p in parameters:
    p.requires_grad = True

In [77]:
lri = []
lossi = []
stepi = []

for i in range(30000):

    #minibatch construct
    ix = torch.randint(0,Xtr.shape[0],(32,))

    # forward pass
    emb = C[Xtr[ix]] #embeddings list for each of the 32x3 examples. [x][z][a] x 32 times. Where [x] is represented by [embA][embB]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1) #(32,300)
    logits = h @ W2 + b2 #(32,len(vocab))
    loss = F.cross_entropy(logits,Ytr[ix])
    print(loss.item())

    #backward pass
    for p in parameters:
        p.grad = None

    loss.backward()

    #update
    # lr = lrs[i]
    lr = 0.05
    for p in parameters:
        p.data += -lr * p.grad 
        
    #track stats
    # lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.item())




62.806087493896484
57.8337516784668
60.68282699584961
56.86506652832031
62.3116455078125
58.269622802734375
58.58024978637695
57.19165802001953
57.850440979003906
57.987674713134766
54.22588348388672
53.626609802246094
53.43309783935547
56.57797622680664
56.18306350708008
55.38933181762695
55.62147903442383
59.74742126464844
58.07901382446289
53.24571990966797
55.64813995361328
58.72465515136719
52.8109016418457
49.209720611572266
53.729061126708984
56.55437088012695
55.111724853515625
53.9506721496582
54.810157775878906
52.10554885864258
55.533206939697266
57.93490982055664
53.99693298339844
51.96961212158203
54.9973258972168
55.03590774536133
49.45732116699219
54.88100814819336
55.55984115600586
54.00482940673828
57.66836929321289
56.00334167480469
55.38081359863281
53.22352981567383
55.435218811035156
51.965396881103516
50.60807800292969
54.10482406616211
53.991416931152344
48.484012603759766
47.59860610961914
53.621707916259766
52.29631805419922
51.4067268371582
56.26231384277344
5

In [107]:
# sampling
block_size = 3

while True:
    out = []
    inputString = ["harry","ran","to"]
    context = [WordsToIndex[inputString[0]],WordsToIndex[inputString[1]],WordsToIndex[inputString[2]]]
    emb = C[torch.tensor([context])] # 1, block_size, d
    h = torch.tanh(emb.view(1,-1) @ W1 + b1)
    logits = h @ W2 + b2
    probs = F.softmax(logits,dim=1)
    ix = torch.multinomial(probs,num_samples=1).item()
    context = context[1:] + [ix]
    # print(f"Context: {context}")
    out.append(ix)
    # print(f"Context: {context}")
    print(IndexToWords[ix])
    time.sleep(.8)

to
wizard
was
lifted
seemed
madam
views
drained
ten
him
swell
undressed
on
on


KeyboardInterrupt: 

In [95]:
print (' '.join(IndexToWords[i] for i in out))

hermione
